In [1]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;
DROP TABLE IF EXISTS posts_sample_external;


-- Create external table 

-- Your code here
CREATE EXTERNAL TABLE posts_sample_external (
    Id string,
    CreationYear string,
    CreationMonth string
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+).*\")(?=.*\\bCreationDate=\"(\\d{4}-\\d+).*\").*' 
    )
LOCATION '/precreate';

Overwriting task1_create_external_table.hql


In [2]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here
SELECT * FROM stackoverflow_.posts_sample_external
LIMIT 10;

Overwriting task1_check_select.hql


In [3]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    Id string
)
PARTITIONED BY (
    CreationMonth string,
    CreationYear string
);

Overwriting task1_create_managed_table.hql


In [4]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table

USE stackoverflow_;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.enforce.bucketing=true;
SET hive.mapred.supports.subdirectories=true;

-- Your code here for inserting data
FROM posts_sample_external pse
INSERT overwrite table posts_sample
PARTITION (CreationMonth, CreationYear)
SELECT pse.Id, pse.CreationMonth, pse.CreationYear;

Overwriting task1_insert_table.hql


In [5]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT * FROM stackoverflow_.posts_sample
LIMIT 10;

Overwriting task1_watch_new_table.hql


In [6]:
%%writefile task1_result.hql
-- Your code here
with cte as (
    select count(*) as total, CreationYear, CreationMonth
    from demodb.posts_sample
    group by CreationYear, CreationMonth
), cte2 as (
    select total, CreationYear, CreationMonth,
        dense_rank() over (order by CreationMonth) as rang
    from cte
)
select CreationYear, CreationMonth, total
from cte2
where rang=3;

Overwriting task1_result.hql


In [7]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [8]:
!cat task1.hql

-- Create external table posts_sample_external with suitable values
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;
DROP TABLE IF EXISTS posts_sample_external;


-- Create external table 

-- Your code here
CREATE EXTERNAL TABLE posts_sample_external (
    Id string,
    CreationYear string,
    CreationMonth string
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+).*\")(?=.*\\bCreationDate=\"(\\d{4}-\\d+).*\").*' 
    )
LOCATION '/precreate';
-- create managed table
-- Check that this table contains info about year and month
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/li

In [9]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [10]:
%%bash
hive -f task1.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.256 seconds
OK
Time taken: 0.347 seconds
OK
Time taken: 0.224 seconds
OK
Time taken: 0.024 seconds
OK
Time taken: 1.258 seconds
OK
Time taken: 0.109 seconds
OK
Time taken: 0.058 seconds
Query ID = jovyan_20211118173044_9bfc5e6a-0bb6-40b2-b3be-eb7f54a0c87d
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1637250744737_0004, Tracking URL = http://172.17.0.27:8088/proxy/application_1637250744737_0004/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1637250744737_0004
Hadoop job information for Stage-1: number of mappers: 0; number of reducers: 0
2021-11-18 17:30:51,144 Stage-1 map = 0%,  reduce = 0%
Ended Job = job_1637250744737_0004
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filter

CalledProcessError: Command 'b'hive -f task1.hql\n'' returned non-zero exit status 17.